### Versi 2

In [1]:
# ===========================================
# Cell 1. Import Library dan Setup Environment
# ===========================================

import os
import json
import random
from pathlib import Path
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import zipfile
import io

# Gunakan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device yang digunakan:", device)


Device yang digunakan: cuda


In [2]:
# ===========================================
# Cell 2. Fungsi Bantuan Umum
# ===========================================

from matplotlib.colors import ListedColormap

def seed_everything(seed=42):
    """Menetapkan seed random agar hasil eksperimen bisa direplikasi"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

def visualize_tile(x_tile, y_true=None, y_pred=None, json_path=None,class_names=None, idx=0):
    """
    Menampilkan citra tile beserta mask ground-truth dan prediksi
    """
    if isinstance(x_tile, torch.Tensor):
        x = x_tile.cpu().numpy()
        x = np.transpose(x, (1,2,0))  # ubah dari [B,H,W] -> [H,W,B]
    else:
        x = x_tile

    # menampilkan pseudo-RGB (karena data hyperspectral)
    B = x.shape[2]
    b1, b2, b3 = int(B*0.05), int(B*0.5), int(B*0.9)
    rgb = x[..., [b1, b2, b3]]
    rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)

    # Coba baca colormap dari file JSON
    if json_path and os.path.exists(json_path):
        with open(json_path, "r") as f:
            label_info = json.load(f)
        custom_colors = [c["color"][:7] for c in label_info]
        cmap = ListedColormap(custom_colors)
    else:
        print("File json tidak terbaca, menggunakan cmap tab20")
        cmap = "tab20"  # fallback

        

    # Visualisasi
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(rgb_norm); plt.title("Citra (Pseudo-RGB)")
    if y_true is not None:
        plt.subplot(1,3,2); plt.imshow(y_true, cmap=cmap); plt.title("Ground Truth")
    if y_pred is not None:
        plt.subplot(1,3,3); plt.imshow(y_pred, cmap=cmap); plt.title("Prediksi")
    plt.show()



In [3]:
# =========================================================
# Cell 3. Dataset Loader (SeaweedDataset) dan Label Mapping
# =========================================================

def load_label_mapping(json_path):
    """Membaca file label_classes.json untuk mapping id ke nama kelas"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    idx_to_name = {i: item["name"] for i, item in enumerate(data)}
    return idx_to_name

# def normalize_reflectance(cube):
#     """Menormalkan nilai reflektansi ke rentang 0-1 per tile"""
#     cube = np.nan_to_num(cube).astype(np.float32)
#     min_val = np.nanmin(cube)
#     max_val = np.nanmax(cube)
#     if max_val > min_val:
#         cube = (cube - min_val) / (max_val - min_val)
#     return cube

def normalize_reflectance(cube):
    """Menormalkan reflektansi 0–1 per tile, hemat RAM, aman untuk mmap read-only."""
    # Jika hasil np.load mmap, array biasanya read-only → buat copy ringan
    if not cube.flags.writeable:
        cube = cube.astype(np.float32, copy=True)  # hanya salin tile, bukan file besar

    # Pastikan float32
    if cube.dtype != np.float32:
        cube = cube.astype(np.float32, copy=False)

    # Ganti NaN / Inf in-place
    np.nan_to_num(cube, copy=False)

    # Normalisasi min-max
    min_val = np.nanmin(cube)
    max_val = np.nanmax(cube)
    if max_val > min_val:
        cube -= min_val
        cube /= (max_val - min_val + 1e-8)

    return cube


class SeaweedDataset(Dataset):
    """
    Dataset hemat memori berbasis file .npy hasil konversi.
    Membaca tile langsung dari disk menggunakan mmap_mode="r".
    """
    def __init__(self, data_files, label_map, tile_size=128, normalize=True, label_remap=None):
        self.data_files = data_files
        self.label_map = label_map
        self.tile_size = tile_size
        self.normalize = normalize
        # label_remap: dict {orig_label: new_index}, if None -> identity mapping
        self.label_remap = label_remap

        # Daftar pasangan (file_x, file_y)
        self.pairs = []
        for f in data_files:
            if f.endswith("_x.npy"):
                fy = f.replace("_x.npy", "_y.npy")
                if os.path.exists(fy):
                    self.pairs.append((f, fy))
        
        # Hanya menyimpan indeks tile berdasarkan ukuran file .npy
        self.index = []  
        for file_idx, (fx, fy) in enumerate(self.pairs):
            x = np.load(fx, mmap_mode="r")
            H, W, _ = x.shape
            for i in range(0, H - tile_size + 1, tile_size):
                for j in range(0, W - tile_size + 1, tile_size):
                    self.index.append((file_idx, i, j))
            del x  # bebaskan referensi memori

        print(f"[INFO] Total tile terdaftar: {len(self.index)} dari {len(self.pairs)} file")

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        file_idx, i, j = self.index[idx]
        fx, fy = self.pairs[file_idx]
        
        # Memuat tile menggunakan mmap
        x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
        y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]

        # Abaikan tile kosong (semua 0) dengan fail-safe agar tidak infinite recursion
        # kalau tetap kosong setelah 3 percobaan → biarkan saja y tetap kosong (model akan skip naturally karena weight=0 utk bg)
        for _ in range(3):  # coba maksimal 3 kali
            if np.any(y > 0):
                break
            file_idx, i, j = self.index[np.random.randint(0, len(self.index))]
            fx, fy = self.pairs[file_idx]
            x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
            y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]


        if self.normalize:
            x = normalize_reflectance(x)

        # REMAP label bila mapping diberikan
        if self.label_remap is not None:
            # buat array output dengan nilai default 0 (background) atau -1 jika ingin ignore
            y_remap = np.zeros_like(y, dtype=np.int64)
            # set default to 0 (background) then map others
            for orig_label, new_idx in self.label_remap.items():
                # gunakan boolean mask assignment (efisien)
                if orig_label == 0:
                    # background -> keep 0 (or explicitly assign)
                    y_remap[y == orig_label] = new_idx
                else:
                    y_remap[y == orig_label] = new_idx
            y = y_remap
        else:
            y = y.astype(np.int64)

        # Konversi ke tensor
        x_tensor = torch.tensor(x.transpose(2, 0, 1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        return x_tensor, y_tensor


def detect_actual_classes(pairs):
    """Scan semua file y.npy untuk mendeteksi kelas yang benar-benar ada"""
    found = set()
    for _, fy in pairs:
        y = np.load(fy, mmap_mode="r")
        found |= set(np.unique(y))
    found = sorted(list(found))
    print(f"[INFO] Kelas AKTUAL yang ditemukan di dataset: {found}")
    return found

In [4]:
# ==============================================
# Cell 4. Load Dataset dan FILE-LEVEL Splitting
# ==============================================

data_dir = "../data/npy_converted"
label_json_path = "../data/annotation/segmentation_masks/label_classes.json"

label_map = load_label_mapping(label_json_path)
print(f"Jumlah total kelas di JSON: {len(label_map)}")

# Ambil semua file _x.npy
all_x_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith("_x.npy")])
pairs = [(fx, fx.replace("_x.npy", "_y.npy")) for fx in all_x_files if os.path.exists(fx.replace("_x.npy", "_y.npy"))]

print(f"Total pasangan file X-Y ditemukan: {len(pairs)} (expected: 18)")
for p in pairs[:5]:
    print("Contoh:", os.path.basename(p[0]), "<->", os.path.basename(p[1]))

# Split deterministik berbasis urutan nama (11 train, 5 val, 3 test)
train_pairs = pairs[:11]
val_pairs   = pairs[11:16]
test_pairs  = pairs[16:]

print("\n=== FINAL SPLIT PER FILE ===")
print(f"Train : {len(train_pairs)}")
print(f"Val   : {len(val_pairs)}")
print(f"Test  : {len(test_pairs)}")

# DETEKSI kelas aktual yang benar-benar muncul (bukan ambil dari JSON)
actual_classes = detect_actual_classes(train_pairs + val_pairs + test_pairs)

# actual_classes adalah list of numpy ints e.g. [0,8,12,13,14,18,38]
orig_classes = [int(x) for x in actual_classes]  # cast to python ints
# Buat mapping orig_label -> contiguous idx 0..(C-1)
label_remap = {orig: idx for idx, orig in enumerate(orig_classes)}
print(f"[INFO] Label remap (orig -> new): {label_remap}")

# Update datasets: pass label_remap ke SeaweedDataset
train_dataset = SeaweedDataset([p[0] for p in train_pairs], label_map, tile_size=128, label_remap=label_remap)
val_dataset   = SeaweedDataset([p[0] for p in val_pairs], label_map, tile_size=128, label_remap=label_remap)
test_dataset  = SeaweedDataset([p[0] for p in test_pairs], label_map, tile_size=128, label_remap=label_remap, normalize=False)


# Hitung frekuensi kelas dari TRAIN dataset (hitung dari file y, lebih efisien)
'''
Menghitung distribusi jumlah pixel dari setiap kelas di TRAIN SET
Lalu menghitung bobot loss yang adil (class weights) berdasarkan distribusi ini
Supaya kelas langka tidak tertindas / diabaikan oleh model, 
karena dataset Imbalance besar (background 0 paling dominan)
'''
from collections import Counter
counter = Counter()
for _, fy in train_pairs:
    y = np.load(fy, mmap_mode="r")
    # remap using label_remap quickly:
    for orig, new in label_remap.items():
        cnt = int((y == orig).sum())
        counter[new] += cnt

print(f"[INFO] Pixel counts per (remapped) class (train set): {dict(counter)}")

# Buat class weights (inverse frequency), dan set weight[0]=0 karena ignore_index=0
counts = np.array([counter.get(i, 0) for i in range(len(label_remap))], dtype=np.float64)
eps = 1e-6
inv_freq = 1.0 / (counts + eps)
# optional normalization so that mean weight = 1
inv_freq = inv_freq / np.mean(inv_freq)
# set background index weight to 0 (ignored)
inv_freq[0] = 0.0

print(f"[INFO] Class weights (remapped, sum-normalized, bg=0): {inv_freq}")

# Simpan nilai-nilai penting ke variabel global untuk digunakan Cell6
num_classes_actual = len(label_remap)
label_remap_global = label_remap
# class_weights_np = inv_freq.astype(np.float32)

# Untuk stabilisasi awal training — pure uniform weights
class_weights_np = np.ones(7, dtype=np.float32)

print(f"\nTotal TILE train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")


Jumlah total kelas di JSON: 41
Total pasangan file X-Y ditemukan: 18 (expected: 18)
Contoh: massimal_smola_maholmen_202306211129-2_hsi_003_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_003_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_004_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_004_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_008_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_008_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_009_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_009_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_013_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_013_processed_y.npy

=== FINAL SPLIT PER FILE ===
Train : 11
Val   : 5
Test  : 2
[INFO] Kelas AKTUAL yang ditemukan di dataset: [np.int32(0), np.int32(8), np.int32(12), np.int32(13), np.int32(14), np.int32(18), np.int32(38)]
[INFO] Label remap (o

In [ ]:
# === CELL DEBUG (BUKAN CELL URUTAN ===

# print(np.load(train_pairs[0][0], mmap_mode="r").shape)
# print(np.load(train_pairs[1][0], mmap_mode="r").shape)
# print(np.load(train_pairs[-1][0], mmap_mode="r").shape)

import torch
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [5]:
# ======================================================
# Cell 5. Model Fully Convolutional HybridSN (3D+2D CNN)
# ======================================================

class FCHybridSN(nn.Module):
    def __init__(self, in_bands=300, num_classes=7): # nilai num_classes dijadikan default 7
        super().__init__()
        self.conv3d_1 = nn.Conv3d(1, 16, (7,3,3), padding=(0,1,1))
        self.bn3d_1 = nn.BatchNorm3d(16)
        self.conv3d_2 = nn.Conv3d(16, 32, (5,3,3), padding=(0,1,1))
        self.bn3d_2 = nn.BatchNorm3d(32)
        self.conv3d_3 = nn.Conv3d(32, 64, (3,3,3), padding=(0,1,1))
        self.bn3d_3 = nn.BatchNorm3d(64)

        self._out_spec = in_bands - 12
        mid_ch = 256
        self.conv2d_1 = nn.Conv2d(64 * max(1, self._out_spec), mid_ch, 3, padding=1)
        self.bn2d_1 = nn.BatchNorm2d(mid_ch)
        self.conv2d_2 = nn.Conv2d(mid_ch, 128, 3, padding=1)
        self.bn2d_2 = nn.BatchNorm2d(128)
        self.conv2d_3 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn2d_3 = nn.BatchNorm2d(64)
        self.classifier = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        B, Bands, H, W = x.shape
        x3 = x.unsqueeze(1)
        x3 = F.relu(self.bn3d_1(self.conv3d_1(x3)))
        x3 = F.relu(self.bn3d_2(self.conv3d_2(x3)))
        x3 = F.relu(self.bn3d_3(self.conv3d_3(x3)))
        B, C3, out_spec, H, W = x3.shape
        x2 = x3.view(B, C3 * out_spec, H, W)
        x2 = F.relu(self.bn2d_1(self.conv2d_1(x2)))
        x2 = F.relu(self.bn2d_2(self.conv2d_2(x2)))
        x2 = F.relu(self.bn2d_3(self.conv2d_3(x2)))
        return self.classifier(x2)

# Ambil jumlah band langsung dari data TRAIN pertama
sample_x = np.load(train_pairs[0][0], mmap_mode="r")
in_bands_actual = sample_x.shape[2]  # ambil jumlah band asli dari npy

print(f"Band input aktual terdeteksi: {in_bands_actual}")

# GUNAKAN jumlah kelas AKTUAL (bukan 41 dari JSON!)
model = FCHybridSN(in_bands=in_bands_actual, num_classes=num_classes_actual).to(device)

print(model)


Band input aktual terdeteksi: 300
FCHybridSN(
  (conv3d_1): Conv3d(1, 16, kernel_size=(7, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_2): Conv3d(16, 32, kernel_size=(5, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_1): Conv2d(18432, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2d_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2d_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1,

In [6]:
# ===========================================
# Cell 6. Keperluan Evaluasi & Loss / Optimizer (diperbarui)
# ===========================================
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# jika class_weights_np sudah uniform ones, tidak perlu pakai weights here.
# (jika nanti ingin pakai safe clipped weights, bisa diubah di sini)
# weight_tensor = torch.from_numpy(class_weights_np)

# enable cudnn benchmark untuk kecepatan (bagus ketika input sizes konstan)
torch.backends.cudnn.benchmark = True

# hyperparameters (adjustable)
LR = 1e-4
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 1   

# ==== Dataloaders ====
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

# Criterion: uniform (tanpa class weights) - stable start
criterion = nn.CrossEntropyLoss(ignore_index=0)

# Optimizer & scheduler (setelah model available)
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

# Scaler ini digunakan saat nanti menggunakan autocast
scaler = None  # placeholder; dan juga tidak digunakan


def pixel_accuracy(pred, target):
    valid = (target >= 0)
    correct = (pred[valid] == target[valid]).sum()
    total = valid.sum()
    return (correct.float() / (total.float() + 1e-9)).item()

def iou_per_class(pred, target, num_classes):
    ious = []
    for cls in range(num_classes):
        pred_i = (pred == cls)
        target_i = (target == cls)
        inter = (pred_i & target_i).sum()
        union = (pred_i | target_i).sum()
        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append((inter.float() / union.float()).item())
    return ious


C:\Users\admin\miniconda3\envs\algae\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [8]:
# Cell periksa 1
import torch, numpy as np
print("Device:", device)
print("CUDA available:", torch.cuda.is_available())
print("num_classes_actual =", num_classes_actual)
print("label_remap =", label_remap)
print("LR =", LR)
print("Criterion:", criterion)
print("scaler is None (AMP off)?:", scaler is None)



Device: cuda
CUDA available: True
num_classes_actual = 7
label_remap = {0: 0, 8: 1, 12: 2, 13: 3, 14: 4, 18: 5, 38: 6}
LR = 0.0001
Criterion: CrossEntropyLoss()
scaler is None (AMP off)?: True


In [10]:
# Cell periksa 2
# ambil beberapa sample dari train_dataset (menggunakan __getitem__ yang sudah meremap)
for idx in range(10):
    x, y = train_dataset[idx]   # x: Tensor [Bands, H, W], y: Tensor [H,W]
    y_np = y.numpy()
    print(f"idx={idx} | x.shape={x.shape} | y.shape={y_np.shape} | unique labels={np.unique(y_np)} | max={y_np.max()} | min={y_np.min()}")


idx=0 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 6] | max=6 | min=0
idx=1 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3 6] | max=6 | min=0
idx=2 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3 6] | max=6 | min=0
idx=3 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3 6] | max=6 | min=0
idx=4 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3 6] | max=6 | min=0
idx=5 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 6] | max=6 | min=0
idx=6 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3] | max=3 | min=0
idx=7 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 3 6] | max=6 | min=0
idx=8 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128) | unique labels=[0 6] | max=6 | min=0
idx=9 | x.shape=torch.Size([300, 128, 128]) | y.shape=(128, 128

In [9]:
# Cell periksa 3
import numpy as np
for fx, fy in train_pairs[:5]:
    y = np.load(fy, mmap_mode="r")
    uniq = np.unique(y)
    print(os.path.basename(fy), "unique raw labels (orig):", uniq)


massimal_smola_maholmen_202306211129-2_hsi_003_processed_y.npy unique raw labels (orig): [ 0 13 14 18 38]
massimal_smola_maholmen_202306211129-2_hsi_004_processed_y.npy unique raw labels (orig): [ 0 12]
massimal_smola_maholmen_202306211129-2_hsi_008_processed_y.npy unique raw labels (orig): [ 0 12]
massimal_smola_maholmen_202306211129-2_hsi_009_processed_y.npy unique raw labels (orig): [ 0 12]
massimal_smola_maholmen_202306211129-2_hsi_013_processed_y.npy unique raw labels (orig): [0 8]


In [11]:
# Cell periksa 4
# dry-run: ambil 20 batch pertama dan cek loss nan
n_batches_check = 20
cnt = 0
for xb, yb in train_loader:
    xb = xb.to(device); yb = yb.to(device)
    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda', enabled=(device.type=='cuda')):
            logits = model(xb)
            loss = criterion(logits, yb)
    print(f"batch {cnt}: loss = {float(loss):.6f}, logits finite? {torch.isfinite(logits).all().item()}")
    if torch.isnan(loss) or not torch.isfinite(loss):
        print("FOUND NAN/INF LOSS in batch", cnt)
        # print some diagnostics
        print(" - preds unique:", torch.unique(logits.argmax(dim=1)).cpu().numpy())
        print(" - labels unique:", torch.unique(yb).cpu().numpy())
        break
    cnt += 1
    if cnt >= n_batches_check:
        break
print("dry-run done")


batch 0: loss = 2.180907, logits finite? True
batch 1: loss = 1.996143, logits finite? True
batch 2: loss = 2.259423, logits finite? True
batch 3: loss = 2.156544, logits finite? True
batch 4: loss = 1.995491, logits finite? True
batch 5: loss = 2.009948, logits finite? True
batch 6: loss = 1.997626, logits finite? True
batch 7: loss = 2.000330, logits finite? True
batch 8: loss = 2.218812, logits finite? True
batch 9: loss = 1.801834, logits finite? True
batch 10: loss = 1.993381, logits finite? True
batch 11: loss = 2.180167, logits finite? True
batch 12: loss = 2.247280, logits finite? True
batch 13: loss = 1.997035, logits finite? True
batch 14: loss = 2.246928, logits finite? True
batch 15: loss = 2.156216, logits finite? True
batch 16: loss = 1.996756, logits finite? True
batch 17: loss = 2.247705, logits finite? True
batch 18: loss = 1.993167, logits finite? True
batch 19: loss = 2.159674, logits finite? True
dry-run done


In [12]:
# Cell periksa 5
from collections import Counter
cnt_bg_batches = 0
for i, (xb, yb) in enumerate(train_loader):
    y_np = yb.numpy()
    if np.all(y_np == 0):
        cnt_bg_batches += 1
    if i >= 200: break
print("Number of all-background batches in first 200 batches:", cnt_bg_batches)


Number of all-background batches in first 200 batches: 34


In [ ]:
# ===========================================
# Cell 7. Loop Training Utama (+ checkpoint) — no AMP + uniform class weights
# ===========================================
import os, time
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.amp import autocast

START_EPOCH = 1     # mulai default 1 (kamu sebelumnya sempat pakai 10)
NUM_EPOCHS = 6     # boleh disesuaikan; start conservative
CLIP_NORM = 5.0 # grad clip untuk stabilitas
best_val_acc = 0.0

checkpoint_path = "hybridsn_sgmt_ver2_checkpoint_stable.pth"
best_model_path = "hybridsn_sgmt_ver2_best_model_stable.pth"

model = model.to(device)

# Load checkpoint jika ada (including scaler & scheduler)
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    scaler.load_state_dict(checkpoint.get("scaler_state", {}))
    START_EPOCH = checkpoint["epoch"] + 1
    best_val_acc = checkpoint.get("best_val_acc", 0.0)
    print(f"[INFO] Memuat checkpoint {checkpoint_path}, resume epoch {START_EPOCH}")
else:
    print("[INFO] Tidak ditemukan checkpoint. Mulai training dari awal.")


# Container untuk plotting/logging
history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": [],
    "val_miou": []
}

# Safety: try to reduce fragmentation (optional)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128")


for epoch in range(START_EPOCH, NUM_EPOCHS + 1):
    start_time = time.time()

    # ---------- TRAIN ----------
    model.train()
    running_loss = 0.0
    running_correct = 0
    running_total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}", leave=True)
    for xb, yb in pbar:
        try:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            
            optimizer.zero_grad()
        
            # Gunakan autocast untuk forward agar hemat memory (FP16)
            with autocast(device_type='cuda', enabled=(device.type == 'cuda')):
                logits = model(xb)               # [B, C, H, W]
                loss = criterion(logits, yb)
        

            # backward + grad clip + step (no AMP)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            optimizer.step()
        
            # akumulasi
            bs = xb.shape[0]
            running_loss += (loss.item() * bs)
            preds = logits.argmax(dim=1)
            valid = (yb >= 0)
            running_correct += (preds[valid] == yb[valid]).sum().item()
            running_total += valid.sum().item()
        
            pbar.set_postfix({"TrainLoss": f"{loss.item():.4f}"})
            
        except RuntimeError as e:
            # Meng-handle potential OOM pada level batch: clear cache and skip batch
            if 'out of memory' in str(e).lower():
                print("[WARN] OOM terjadi saat batch forward/backward. Clear cache dan skip batch.")
                torch.cuda.empty_cache()
                optimizer.zero_grad()
                continue
            else:
                # kirim exceptions lain
                raise
    avg_train_loss = running_loss / max(1, len(train_loader.dataset))
    train_acc = running_correct / (running_total + 1e-9)

    # ---------- VALIDATION ----------
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    val_running_total = 0
    val_ious = []
    
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            
            # autocast untuk validasi forward
            with autocast(device_type='cuda', enabled=(device.type == 'cuda')):
                logits = model(xb)
                loss = criterion(logits, yb)
            
            val_running_loss += loss.item() * xb.shape[0]
            preds = logits.argmax(dim=1)
            
            valid = (yb >= 0)
            val_running_correct += (preds[valid] == yb[valid]).sum().item()
            val_running_total += valid.sum().item()
            val_ious.extend(iou_per_class(preds, yb, num_classes_actual))

    avg_val_loss = val_running_loss / max(1, len(val_loader.dataset))
    val_acc = val_running_correct / (val_running_total + 1e-9)
    mean_iou = np.nanmean([v for v in val_ious if not np.isnan(v)])
    
    # Scheduler step (menggunakan val_acc sebagai metric)
    scheduler.step(val_acc)

    # Logging & checkpoint
    history["train_loss"].append(avg_train_loss)
    history["val_loss"].append(avg_val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)
    history["val_miou"].append(mean_iou)

    elapsed = time.time() - start_time

    print(f"Epoch {epoch}/{NUM_EPOCHS} | TrainLoss={avg_train_loss:.4f} | ValLoss={avg_val_loss:.4f} | TrainAcc={train_acc:.4f} | ValAcc={val_acc:.4f} | mIoU={mean_iou:.4f} | Time={elapsed/3600:.2f} jam")

    # Checkpoint (tanpa simpan scaler state)
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_val_acc": best_val_acc,
    }
    torch.save(checkpoint, checkpoint_path)

    # Save best model by val_acc
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(checkpoint, best_model_path)
        print("[INFO] Model terbaik disimpan.")

# ---------- AFTER ALL EPOCHS: PLOT LOSS ----------
plt.figure(figsize=(8,4))
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# clear GPU cache
torch.cuda.empty_cache()